#### DO views the pending requests and takes an action

The Data Owner can view the requests that are submitted to the domain. Following is the information visible to the user:

For Data Requests

- Request Id (Unique id of the request)
- Request Date (Datetime on which the request was submitted. The datetime/timestamp are shown in UTC)
- Reason (The reason submitted to access the resource by requester)
- Result Id (The unique id of the reasource being requested)
- State (State of the request - Approved/Declined/Pending)
- Reviewer Comments

For Budget Requests

- Request Id (Unique id of the request)
- Request Date (Datetime on which the request was submitted. The datetime/timestamp are shown in UTC)
- Reason (The reason submitted to access the resource by requester)
- Current Budget (Current Private Budget)
- Requested Budget (The number of epsilons requested by the user)
- State (State of the request - Approved/Denied/Pending)

A DO user can perform the following operations: **[P0]**
- List the requests submitted to the domain
- Select a request via `index` or `Request Id`
- Take action on the request (Available options are: Approve/Decline)
- Provide an optional `comment` on approval or decline by the Domain Owner.


**Note**
Currently, the requests are not persisitent. They are deleted, as soon as an action (approve/deny) is taken on them.

In [ ]:
import syft as sy

In [ ]:
# Let's login into my domain

rnsa_domain_client = sy.login(
    url="https://rsna.openmined.org",
    email="info@openmined.org", password="bazinga", 
    port=8081
)

#### Approve/Deny Data Requests

In [3]:
# Let's have a look for an requests pending for review on the domain
# We can call the `.requests` method to list all the pending requests
# Note: `.requests` will only list the requests with state == `Pending`
rsna_domain_client.requests

You have 1 pending data request


,Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,65cb262f68dd4316890eb6a2440a4d5a,Sep 29 2021 11:38AM,Know the number of unique labels in the dataset.,4df1108de08c4557b50fc53f49011c10,Pending,-


In [4]:
# Let's say we want to list all the requests that have been submitted to the domain
# And not just the pending requests, we can do as:
# `requests.all()`

rnsa_domain_client.requests.all()

,Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,65cb262f68dd4316890eb6a2440a4d5a,Sep 29 2021 11:38AM,Know the number of unique labels in the dataset.,4df1108de08c4557b50fc53f49011c10,Pending,-
1,609fb84f10714d0ab93197980bea0a2c,Sep 19 2021 01:38AM,Get the labels in the dataset.,b82cc4050c9c438c9a1b1f9474b81082,Denied,Access to raw labels is not allowed
2,2eec3515210b4fbab58a903ddcfa7e8e,Sep 09 2021 01:38AM,Get the number of columns in the dataset.,6781046e93d9482381bf0f0d4016a4d2,Approved,-


In [12]:
# Let's select a pending request and take some action
# A user can select a request object via `index` or `id`

# Let's say we pick up the first request related to privacy budget
request_obj = rsna_domain_client.requests[0]

# or via "Request Id"

request_obj = rsna_domain_client.requests["65cb262f68dd4316890eb6a2440a4d5a"]

In [17]:
# The user can either `approve` or `deny` a request

request_obj.approve(comment="Look good for now")

# Or `deny` the request 

request_obj.deny(comment="Information not allowed")

# Let's check for pending requests
rsna_domain_client.requests

You have no pending requests.


In [5]:
# Let's say the last request was approved
# and we want to list all the requests submitted to the domain so far
rsna_domain_client.requests.all()

,Id,Request Date,Reason,Result Id,State,Reviewer Comments
0,65cb262f68dd4316890eb6a2440a4d5a,Sep 29 2021 11:38AM,Know the number of unique labels in the dataset.,4df1108de08c4557b50fc53f49011c10,Approved,Look good for now
1,609fb84f10714d0ab93197980bea0a2c,Sep 19 2021 01:38AM,Get the labels in the dataset.,b82cc4050c9c438c9a1b1f9474b81082,Denied,Look good for now
2,2eec3515210b4fbab58a903ddcfa7e8e,Sep 09 2021 01:38AM,Get the number of columns in the dataset.,6781046e93d9482381bf0f0d4016a4d2,Approved,Look good for now


#### Approve/Deny Budget requests

In [11]:
# Let's check if there are any pending budget requests
# We can list all the `Pending` requests as:
rsna_domain_client.pb_requests

You have 1 pending budget request


,Request Id,Request Date,Reason,Current Budget,Requested Budget,State
0,89479321bb96468186eac85cde50a925,Sep 26 2021 05:53PM,Need more budget since I need to train a model.,2ε,10ε,Pending


In [ ]:
# We can select the budget request object either via `Request Id` or `index`

# via `index`
pb_request_obj = rsna_domain_client.pb_requests[0]

# Or via `Request Id`

pb_request_obj = rsna_domain_client.pb_requests["89479321bb96468186eac85cde50a925"]

In [ ]:
# We can approve/deny the request as follows:

# Approve the request
pb_request_obj.approve()

# Or deny the request
pb_request_obj.deny()

In [22]:
# Let's say we declined the budget request, then if we check the for pending requests, it should be empty.
rsna_domain_client.pb_requests

No pending requests.


In [12]:
# But, if we list all the budget requests submitted to the domain,
rsna_domain_client.pb_requests.all()

,Request Id,Request Date,Reason,Current Budget,Requested Budget,State
0,89479321bb96468186eac85cde50a925,Sep 26 2021 05:53PM,Need more budget since I need to train a model.,2ε,10ε,Declined
1,e607061479b14451b2c56015f27125f1,Sep 11 2021 05:53PM,Need more budget. Drained out of budget.,0.5ε,2ε,Approved


#### Dummy Data

In [1]:
import pandas as pd
from enum import Enum
import uuid
import torch
import datetime
import json
import numpy as np


class bcolors(Enum):
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"


request_budget_id = uuid.uuid4().hex

requests_data = [
    {
        "Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().strftime("%b %d %Y %I:%M%p"),
        "Reason": "Know the number of unique labels in the dataset.",
        "Result Id": uuid.uuid4().hex,
        "State": "Pending",
        "Reviewer Comments": "-",
    },
    {
        "Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().replace(day=19, hour=1).strftime("%b %d %Y %I:%M%p"),
        "Reason": "Get the labels in the dataset.",
        "Result Id": uuid.uuid4().hex,
        "State": "Denied",
        "Reviewer Comments": "Access to raw labels is not allowed",
    },
    {
        "Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().replace(day=9, hour=1).strftime("%b %d %Y %I:%M%p"),
        "Reason": "Get the number of columns in the dataset.",
        "Result Id": uuid.uuid4().hex,
        "State": "Approved",
        "Reviewer Comments": "-",
    }
]

requests_data_df = pd.DataFrame(requests_data)


budget_request = [
    {
        "Request Id": request_budget_id,
        "Request Date": datetime.datetime.now().strftime("%b %d %Y %I:%M%p"),
        "Reason": "Need more budget since I need to train a model.",
        "Current Budget": "2ε",
        "Requested Budget": "10ε",
        "State": "Pending",
    },
]

budget_request_df = pd.DataFrame(budget_request)


approved_budget_request = [
    {
        "Request Id": request_budget_id,
        "Request Date": datetime.datetime.now().strftime("%b %d %Y %I:%M%p"),
        "Reason": "Need more budget since I need to train a model.",
        "Current Budget": "2ε",
        "Requested Budget": "10ε",
        "State": "Declined",
    },
    {
        "Request Id": uuid.uuid4().hex,
        "Request Date": datetime.datetime.now().replace(day=11).strftime("%b %d %Y %I:%M%p"),
        "Reason": "Need more budget. Drained out of budget.",
        "Current Budget": "0.5ε",
        "Requested Budget": "2ε",
        "State": "Approved",
    },

]

approved_budget_request_df = pd.DataFrame(approved_budget_request)

In [2]:
approved_data_df = requests_data_df.copy()
approved_data_df["State"][0] = "Approved"
approved_data_df["Reviewer Comments"] = "Look good for now"